In [ ]:
#data and code ideas source:
#https://www.kaggle.com/getting-started/186418

In [1061]:
import pandas as pd
import numpy as np
df=pd.read_csv('result_final.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2190 entries, 0 to 2189
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     2190 non-null   int64 
 1   Unnamed: 0.1   2190 non-null   int64 
 2   link           2190 non-null   object
 3   text           2184 non-null   object
 4   title          2187 non-null   object
 5   date           1691 non-null   object
 6   keywords       2187 non-null   object
 7   summary        2184 non-null   object
 8   title_summary  2184 non-null   object
dtypes: int64(2), object(7)
memory usage: 154.1+ KB


In [1062]:
df.drop_duplicates(['title'],inplace=True)

In [1063]:
df.isnull().sum()

Unnamed: 0         0
Unnamed: 0.1       0
link               0
text               4
title              1
date             396
keywords           1
summary            4
title_summary      4
dtype: int64

In [1064]:
df.rename(columns = {'Unnamed: 0':'news_id'}, inplace = True)
df['date']=pd.to_datetime(df['date'])

In [1065]:
news_df=df[['link','text','title','date']].copy()

In [1066]:
news_df=news_df.reset_index()

In [1067]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1724 entries, 0 to 1723
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   1724 non-null   int64 
 1   link    1724 non-null   object
 2   text    1720 non-null   object
 3   title   1723 non-null   object
 4   date    1328 non-null   object
dtypes: int64(1), object(4)
memory usage: 67.5+ KB


In [1068]:
news_df.isna().sum()

index      0
link       0
text       4
title      1
date     396
dtype: int64

In [1069]:
no_text_title_indices=news_df[(news_df['text'].isnull())&(news_df['title'].isnull())].index

news_df.drop(no_text_title_indices,axis=0,inplace=True)

title_only_indices=news_df[(news_df['text'].isnull())&(news_df['title'].notnull())].index

In [1070]:
for i in title_only_indices:
    news_df.loc[i,'text']=news_df.loc[i,'title']

In [1071]:
def convertlower(s):
    return s.lower()

In [1072]:
news_df['text']=news_df['text'].apply(convertlower)

In [1073]:
def removeWhiteSpaces(s):
    return s.strip()

In [1074]:
news_df['text']=news_df['text'].apply(removeWhiteSpaces)

In [1075]:
import string
def removePunctuations(s):
    return s.translate(str.maketrans('','',string.punctuation))

In [1076]:
news_df['text']=news_df['text'].apply(removePunctuations)

In [1077]:
import re
def removeNumbers(s):
    return re.sub(r'\d+','',s)
def removeURL(s):
    return re.sub(r'^https?:\/\/.*[\r\n]*','',s)
def removeTags(s):
    remove_tags=re.compile('<.*?>')
    return re.sub(remove_tags,'',s)

In [1078]:
news_df['text']=news_df['text'].apply(removeNumbers)
news_df['text']=news_df['text'].apply(removeURL)
news_df['text']=news_df['text'].apply(removeTags)

In [1079]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
tfidf=TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS,max_df=0.8,min_df=0.25)

In [1080]:
tfidf_matrix=tfidf.fit_transform(news_df['text'])

In [1081]:
tfidf_matrix.shape

(1723, 24)

In [1082]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim=linear_kernel(tfidf_matrix,tfidf_matrix)

In [1083]:
cosine_sim

array([[1.        , 0.73882936, 0.41983446, ..., 0.39548028, 0.38090074,
        0.31937249],
       [0.73882936, 1.        , 0.30754811, ..., 0.4576303 , 0.52170967,
        0.43573242],
       [0.41983446, 0.30754811, 1.        , ..., 0.36806687, 0.0538922 ,
        0.20813156],
       ...,
       [0.39548028, 0.4576303 , 0.36806687, ..., 1.        , 0.60160032,
        0.75912578],
       [0.38090074, 0.52170967, 0.0538922 , ..., 0.60160032, 1.        ,
        0.53500493],
       [0.31937249, 0.43573242, 0.20813156, ..., 0.75912578, 0.53500493,
        1.        ]])

In [1084]:
def getRecommendations(newsindex):
    return newsdf.loc[newsindex]

In [1085]:
def getnewsId(newsTitle):
    return news_df.loc[news_df['title']==newsTitle,'index'].values[0]

In [1086]:
recommendations_count=10
def getRecommendations(read_id):
    return np.argsort(cosine_sim[read_id][:])[::-1][1:recommendations_count+1]


In [1087]:
def newsPrettyPrint(result_df):
    for i in range(recommendations_count):
        print('Recommendation '+str((i+1))+'\n')
        print('Date :',result_df.loc[i,'date'])
        print('Title :',result_df.loc[i,'title'])
        print('Link :',result_df.loc[i,'link'])
        print('Similarity score:',result_df.loc[i,'score'])
        print('Details :',result_df.loc[i,'text'])
        print('\n')

In [1088]:
def getTopRecommendationsByContent(result_df):
    newsPrettyPrint(result_df)

In [1089]:
def getTopRecommendationsLatestFirst(result_df):
    result_df['date'] = pd.to_datetime(result_df['date'], errors='coerce',utc=True)
    result_df.sort_values(by='date',ascending=False,inplace=True)
    newsPrettyPrint(result_df)

In [1090]:
read_article='Groww, an investment app for millennials in India, raises $30M led by YC Continuity – TechCrunch'
read_id=getnewsId(read_article)

In [1091]:
recommendation_indices=getRecommendations(read_id)
scores_list=[cosine_sim[read_id][i] for i in recommendation_indices]
result_df=news_df.loc[recommendation_indices].copy()
result_df['score']=scores_list
result_df=result_df.reset_index()
print('user read title: \n')
print(read_article)
print('\n')
print("Recommendation by content similarity\n")
getTopRecommendationsByContent(result_df)
print("Recommendation by content similarity date sorted\n")
getTopRecommendationsLatestFirst(result_df)

user read title: 

Groww, an investment app for millennials in India, raises $30M led by YC Continuity – TechCrunch


Recommendation by content similarity

Recommendation 1

Date : 2020-09-15 00:00:00
Title : Virtual events platform Airmeet raises $12M – TechCrunch
Link : http://techcrunch.com/2020/09/15/virtual-events-platform-airmeet-raises-12m/
Similarity score: 0.8678524415065821
Details : airmeet a startup that offers a platform to host virtual events said on tuesday it has raised  million in a new financing round as the bangaloreheadquartered firm demonstrates accelerating growth in its user base

sequoia capital india led the  million series a financing round in oneyearold airmeet redpoint ventures and existing investors accel india venture highway global founders capital gfc and gokul rajaram caviar lead at doordash also participated in the round

the new round values airmeet at about  million more than double of what it was valued in march when it raised  million according to 